In [4195]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, auc,classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast
import os
import pickle
import copy
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, Adafactor
from sklearn.utils.class_weight import compute_class_weight
import time
#from babel.dates import format_time
import re
import math
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
device = torch.device("cuda")
os.getcwd()

'C:\\Users\\Dani1.OFER3090PC\\Documents\\Tal Folder\\thesis\\updated files\\most updated files'

In [4196]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dani1.OFER3090PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4197]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dani1.OFER3090PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4198]:
import os
os.chdir('C:\\Users\\Dani1.OFER3090PC\\Documents\\Tal Folder\\thesis\\updated files\\most updated files')

In [4199]:
torch.cuda.is_available()

True

In [4200]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):

        super(BERT_Arch, self).__init__()

        self.bert = bert 

        # dropout layer
        self.dropout = nn.Dropout(0.1)

        # relu activation function
        self.relu =  nn.ReLU()

        # dense layer 1
        self.fc1 = nn.Linear(768,512)

        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512,2)

        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

        #pass the inputs to the model  
        output = self.bert(sent_id, attention_mask=mask)
       #print(cls_hs)
        #output = output["pooler_output"]
        output = output[1]
                
        x = self.fc1(output)

        x = self.relu(x)

        x = self.dropout(x)

        # output layer
        x = self.fc2(x)

        # apply softmax activation
        x = self.softmax(x)

        return x

In [4201]:
#function to train the model
def train(model,train_dataloader, cross_entropy, optimizer ):
  
    model.train()

    total_loss, total_accuracy = 0, 0

    # empty list to save model predictions
    total_preds=[]

    # iterate over batches
    for step,batch in enumerate(train_dataloader):

        # progress update after every 50 batches.
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        # push the batch to gpu
        batch = [r.to(device) for r in batch]

        sent_id, mask, labels = batch

        # clear previously calculated gradients 
        model.zero_grad()        
        # get model predictions for the current batch
        preds = model(sent_id, mask)

        # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels)

        # add on to the total loss
        total_loss = total_loss + loss.item()

        # backward pass to calculate the gradients
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()

        # model predictions are stored on GPU. So, push it to CPU
        preds=preds.detach().cpu().numpy()

        # append the model predictions
        total_preds.append(preds)
        #torch.cuda.empty_cache()

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    #returns the loss and predictions
    return avg_loss, total_preds

In [4202]:
def evaluate(model, val_dataloader, cross_entropy):
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []
  
  t0 = time.time()

  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      #elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
        preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
        loss = cross_entropy(preds,labels)

        total_loss = total_loss + loss.item()

        preds = preds.detach().cpu().numpy()

        total_preds.append(preds)
      #torch.cuda.empty_cache()  

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds


In [4203]:
datasets_names = [
                   'Kaggle',
                   'Founta',
                    'Razavi',
                   'Waseem',
                   'Kumar',
                   #'RoastMe',
                    'Offensive_Reddit_new'
                  ]
datasets_dict = {}
for datasets_name in datasets_names:
     path = datasets_name+'_Data_balanced.csv'
     dataset = pd.read_csv(path)
     datasets_dict[datasets_name] = dataset
    

In [4204]:
#datasets_dict['Waseem'].columns

In [4205]:
#datasets_dict

In [4206]:
stop_words = stopwords.words("english")
wordnet = WordNetLemmatizer()

In [4207]:
def text_preproc(x):
  x = x.lower()
  x = ' '.join([word for word in x.split(' ') if word not in stop_words])
  x = x.encode('ascii', 'ignore').decode()
  x = re.sub(r'https*\S+', ' ', x)
  x = re.sub(r'@\S+', ' ', x)
  x = re.sub(r'#', ' ', x)
  x = re.sub(r'\'\w+', '', x)
  x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
  x = re.sub(r'\w*\d+\w*', '', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x

In [4208]:
def clean_dataset(dataset):
    df = pd.DataFrame()
    df['clean_text'] = dataset.comment.apply(text_preproc)
    dataset['clean_comment'] = df['clean_text']
    col_list = list(dataset)
    col_list[2], col_list[3] = col_list[3], col_list[2]
    dataset = dataset[col_list]
    return dataset

In [4209]:
for dataset_name in datasets_names: 
    datasets_dict[dataset_name] = clean_dataset(datasets_dict[dataset_name])
#datasets_dict['Kaggle']

In [4210]:
# train_dataset = pd.read_csv('train_dataset_1_Kaggle.csv')
# train_dataset = clean_dataset(train_dataset)
# len(train_dataset)

In [4211]:
#datasets_dict['Offensive_Reddit_new']

In [4212]:
# col_list = list(datasets_dict['Waseem'])
# col_list[2], col_list[3] = col_list[3], col_list[2]
# datasets_dict['Waseem'] = datasets_dict['Waseem'][col_list]
# datasets_dict['Waseem']

In [4213]:
#df.to_csv('clean_waseem.csv',index=False)

In [4214]:
def run_bert(train_text,train_labels,val_text,val_labels,test_text, test_labels):
    bert = AutoModel.from_pretrained('bert-base-uncased')    
    # Load the BERT tokenizer
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')    
    # sample data
    text = ["this is a bert model tutorial", "we will fine-tune a bert model"]
    # encode text
    sent_id = tokenizer.batch_encode_plus(text, padding=True, return_token_type_ids=False)
    print(sent_id)
# tokenize and encode sequences in the training set
    max_seq_len = 180
    tokens_train = tokenizer.batch_encode_plus(
        train_text.tolist(),
        max_length = max_seq_len,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False
        )    
    
    tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
    )
    
    tokens_test= tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
    )
    # for train set
    train_seq = torch.tensor(tokens_train['input_ids'])
    train_mask = torch.tensor(tokens_train['attention_mask'])
    train_y = torch.tensor(train_labels.tolist())

# for validation set
    val_seq = torch.tensor(tokens_val['input_ids'])
    val_mask = torch.tensor(tokens_val['attention_mask'])
    val_y = torch.tensor(val_labels.tolist())

# for test set
    test_seq = torch.tensor(tokens_test['input_ids'])
    test_mask = torch.tensor(tokens_test['attention_mask'])
    test_y = torch.tensor(test_labels.tolist())

    len(test_mask)


#define a batch size
    batch_size = 4

# wrap tensors
    train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
    train_sampler = RandomSampler(train_data)

# dataLoader for train set
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
    val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
    val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
    val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

# freeze all the parameters
    #print(bert.parameters().size())

    #length = 0
#     for param in bert.parameters():
#         length += 1
#         param.requires_grad = False
#     print("parameters length: ")
#     print(length)
#     for name, param in bert.named_parameters(): 
#         if param.requires_grad == True:
#             print(name)
    for name, param in list(bert.named_parameters())[:101]: 
        #print('I will be frozen: {}'.format(name)) 
        param.requires_grad = False
#     for name, param in bert.named_parameters():
#         if param.requires_grad == True:
#             print(name) 
    


# pass the pre-trained BERT to our define architecture
    model = BERT_Arch(bert)
# push the model to GPU
    model = model.to(device)
# define the optimizer
    #optimizer = AdamW(model.parameters(), lr = 1e-4)
    optimizer = Adafactor(model.parameters(),lr = 2e-5, relative_step =False)
    
#compute the class weights
    #class_wts = compute_class_weight(dict(zip(np.unique(train_labels), train_labels)))
    class_wts = compute_class_weight(class_weight = 'balanced', classes = np.unique(train_labels), y =train_labels)
    #class_wts = dict(zip(np.unique(train_labels), class_wts))

    print(class_wts)
# convert class weights to tensor
    weights= torch.tensor(class_wts,dtype=torch.float)
    weights = weights.to(device)

# loss function
    cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
    epochs = 4


    best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
    train_losses=[]
    valid_losses=[]

#for each epoch
    torch.cuda.empty_cache()
    for epoch in range(epochs):
     
        print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
        train_loss, train_preds = train(model, train_dataloader, cross_entropy, optimizer )
    
    #evaluate model
        valid_loss, _ = evaluate(model, val_dataloader,cross_entropy)
    
    #save the best model
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            #torch.save(model.state_dict(), '/kaggle/working/Bert_Models/Kaggle/1/saved_weights_Kaggle.pt')
            #torch.save(model.state_dict(), '/kaggle/input/data-cross-domain/saved_weights_Kaggle.pt')
            final_model = model
    
    # append training and validation loss
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
    
        print(f'\nTraining Loss: {train_loss:.3f}')
        print(f'Validation Loss: {valid_loss:.3f}')
        #torch.cuda.empty_cache()
    torch.cuda.empty_cache()
    
    #load weights of best model saved_model_Kaggle_1.pt
    #path = '/kaggle/input/data-cross-domain/saved_weights_Kaggle.pt'
    #output_model_dir = 'C:\\Users\\Dani1.OFER3090PC\\Documents\\Tal Folder\\thesis\\updated files\\Bert_Models\\Kaggle\\1'
    #model.load_state_dict(torch.load(path))
#     model_to_save = (
#     model.module if hasattr(model, "module") else model
#     )  # Take care of distributed/parallel training
    #model_to_save.save_pretrained(output_model_dir)
    #tokenizer.save_pretrained(output_model_dir)
#     torch.save(model_to_save.state_dict(),output_model_dir)
    test_mask
    
    #torch.cuda.empty_cache()
    
    with torch.no_grad():
        preds_test = final_model(test_seq.to(device), test_mask.to(device))
        preds_test = preds_test.detach().cpu().numpy()
        #preds_train = preds_train.detach().cpu().numpy()
        
    # model's performance
    preds_test = np.argmax(preds_test, axis = 1)
    Report = classification_report(test_y, preds_test, output_dict=True )
    print(Report)

    # confusion matrix
    print(pd.crosstab(test_y, preds_test))
    confusion_matrix = pd.crosstab(test_y, preds_test)
    
    FP = []
    FN = []
    TN = []
    TP = []
    for i in range(len(test_y)):
        if test_y[i] == 0 and  preds_test[i] ==1:
            FP.append(test_text[i])
        elif test_y[i] == 1 and  preds_test[i] ==0:
            FN.append(test_text[i])
        elif test_y[i] == 0 and  preds_test[i] ==0:
            TN.append(test_text[i])
        elif test_y[i] == 1 and  preds_test[i] ==1:
            TP.append(test_text[i])         
#     print("10 False Positives: \n")
#     print(FP[0:11])
#     print("10 False Negatives: \n")
#     print(FN[0:11]) 
#     print("10 True Negatives: \n")
#     print(TN[0:11])
#     print("10 True Positives: \n")
#     print(TP[0:11]) 
    return Report,confusion_matrix, FP, FN, TN, TP, preds_test
    

In [4215]:
def unique(list1):
 
    # initialize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

In [4216]:
def create_balanced_datasets(max_size,dataset_label_0,dataset_label_1):
    #if len(dataset_label_0) + len(dataset_label_1) >  2*max_size:
        minority_len = min(len(dataset_label_0),len(dataset_label_1))
        if  minority_len > max_size:
            balanced_0 = dataset_label_0.sample(max_size)
            balanced_1 = dataset_label_1.sample(max_size)
        else:
            balanced_0 = dataset_label_0.sample(minority_len)
            balanced_1 = dataset_label_1.sample(minority_len)
        balanced_dataset = pd.concat([balanced_0,balanced_1],axis=0,ignore_index=True)
        balanced_dataset = balanced_dataset.sample(frac=1).reset_index(drop=True)
        balanced_dataset['id'] = [i for i in range(1,len(balanced_dataset)+1)]
        #balanced_dataset = shuffle(balanced_dataset)
        return balanced_dataset

In [4217]:
def create_imbalanced_datasets(majority_rate,dataset_label_0,dataset_label_1):
    #if len(dataset_label_0) + len(dataset_label_1) >  2*max_size:
        majority_len = len(dataset_label_0)
        minority_len  = int(np.ceil(majority_len/majority_rate)) - majority_len
        imbalanced_0 = dataset_label_0.sample(majority_len)
        imbalanced_1 = dataset_label_1.sample(minority_len)
        imbalanced_dataset = pd.concat([imbalanced_0,imbalanced_1],axis=0,ignore_index=True)
        imbalanced_dataset = imbalanced_dataset.sample(frac=1).reset_index(drop=True)
        imbalanced_dataset['id'] = [i for i in range(1,len(imbalanced_dataset)+1)]
        #balanced_dataset = shuffle(balanced_dataset)
        return imbalanced_dataset

In [4218]:
def create_imbalnced_from_balanced(n,majority_rate,train_dataset, train_dataset_name):
    for i in range(n):
        train_dataset = pd.read_csv('train_train_dataset_'+ str(i+1)+'_'+train_dataset_name+'.csv')
        train_dataset = clean_dataset(train_dataset) 
        train_dataset = create_imbalanced_datasets(majority_rate,train_dataset[train_dataset['Label'] == 0],train_dataset[train_dataset['Label'] == 1])
        validation_dataset = pd.read_csv('validation_train_dataset_'+ str(i+1)+'_'+train_dataset_name+'.csv')
        validation_dataset = clean_dataset(validation_dataset) 
        validation_dataset = create_imbalanced_datasets(majority_rate,validation_dataset[validation_dataset['Label'] == 0],validation_dataset[validation_dataset['Label'] == 1])
        train_dataset.to_csv('/kaggle/working/train_train_dataset_'+str(i+1)+'_'+train_dataset_name + '_imbalanced.csv',index=False)
        validation_dataset.to_csv('/kaggle/working/validation_train_dataset_'+str(i+1)+'_'+train_dataset_name + '_imbalanced.csv',index=False)
        

In [4219]:
#create_imbalnced_from_balanced(4,0.7,datasets_dict['Razavi'],'Razavi')

In [4220]:
def test_n_times_on_other_dataset(n,train_dataset_name_1,train_dataset_name_2,test_dataset,val_rate,train_dataset_number):
    f1_1 = 0
    f1_weighted = 0
    FP, FN, TN, TP = [],[],[],[]
    preds = []
    best_f1 = 0
    #dataset_number = 1
    #n=7
    for i in range(n):
        #train_dataset = pd.read_csv('train_dataset_'+ str(train_dataset_number)+'_'+train_dataset_name_1+'_'+train_dataset_name_2+'.csv')
        train_dataset = pd.read_csv('train_dataset_'+ str(train_dataset_number)+'_'+train_dataset_name_1+'_RM.csv')
        train_dataset = clean_dataset(train_dataset)
#         validation_dataset = pd.read_csv('validation_dataset_'+ str(dataset_number)+'_'+train_dataset_name+'_RM.csv')
#         validation_dataset = clean_dataset(validation_dataset)
        #train_dataset = create_balanced_datasets(1500,train_dataset[train_dataset['Label']==0],train_dataset[train_dataset['Label']==1])
        #test_dataset = create_balanced_datasets(1000,test_dataset[test_dataset['Label']==0],test_dataset[test_dataset['Label']==1])
        train_text, val_text, train_labels, val_labels = train_test_split(train_dataset['clean_comment'],
                                                         train_dataset['Label'],test_size=val_rate, stratify = train_dataset['Label'])
#         train_text, val_text, train_labels, val_labels = train_test_split(train_dataset['comment'], train_dataset['Label'], 
#                                                                     #random_state=2018,
#                                                                     test_size= 1-train_rate,
#                                                                     stratify=train_dataset['Label'])
#         _, val_text, _, val_labels = train_test_split(rest_text, rest_labels, 
#                                                                     random_state=2018,
#                                                                     test_size=val_rate,
#                                                                     stratify=rest_labels)
#         _, test_text, _, test_labels  = train_test_split(test_dataset['comment'], test_dataset['Label'], 
#                                                                     random_state=2018,
#                                                                     test_size=test_rate,
#                                                                     stratify=test_dataset['Label']
#                                                                     )
        print("train length: " + str(len(train_text)))
        print(np.sum(val_labels == 0))
        print("train length 0 label: " + str(np.sum(train_labels == 0)))
        print("train length 1 label: " + str(np.sum(train_labels == 1)))
        print("val length: " + str(len(val_text)))
        print("val length 0 label: " + str(np.sum(val_labels == 0)))
        print("val length 1 label: " + str(np.sum(val_labels == 1)))
        print("test length: " + str(len(test_dataset['comment'])))
        print("test length 0 label: " + str(len(test_dataset['Label'][test_dataset['Label']==0])))
        print("test length 1 label: " + str(len(test_dataset['Label'][test_dataset['Label']==1])))
        
        #torch.cuda.empty_cache()
        report,confusion_matrix, FP_new, FN_new, TN_new, TP_new, preds_test  = run_bert(train_text,train_labels,val_text,val_labels,test_dataset['clean_comment'], test_dataset['Label'])
        f1_1 += report['1']['f1-score']
        f1_weighted += report['weighted avg']['f1-score']
        if report['weighted avg']['f1-score'] > best_f1:
            best_f1 = report['weighted avg']['f1-score']
            best_report = report
            best_conf_mat = confusion_matrix
        FP = FP + FP_new
        FN = FN + FN_new
        TN = TN + TN_new
        TP = TP + TP_new
        #preds = preds + preds_test
    f1_1 = f1_1/n
    f1_weighted = f1_weighted/n
    
    print(f1_1)
    #test_dictionary['f1_1'] = f1_1
    print(f1_weighted)
    FP = unique(FP)
    FN = unique(FN)
    TN = unique(TN)
    TP = unique(TP)
    #test_dictionary['f1_weighted'] = f1_weighted
    
    return best_report,best_conf_mat, f1_1, f1_weighted, FP, FN, TN, TP, preds_test


In [4221]:
def test_for_train_dataset(train_dataset_name_1,train_dataset_name_2,datasets_dict,n_iter,val_rate,train_dataset_number, test_dataset_number):
    test_dict = {}
    #train_dataset = datasets_dict[train_dataset_name]

    for dataset_name in datasets_dict:
        #if dataset_name == train_dataset_name_1 or dataset_name == train_dataset_name_2:
        if dataset_name == train_dataset_name_1:
        #if dataset_name != 'Founta':
            continue
        else:
            print("train on " + train_dataset_name_1 + " dataset and " + train_dataset_name_2 + ":\n")  
            print("test on: " + dataset_name)
            test_dict[dataset_name] = {}
            test_dict[dataset_name]['dataset'] = datasets_dict[dataset_name]
            #path = 'train_dataset'+'_'+ str(test_dataset_number)+'_'+ dataset_name +'.csv'
            path = 'train_dataset'+'_'+ str(test_dataset_number)+'_'+ dataset_name +'.csv'
            test_dataset = pd.read_csv(path)
            #test_dataset = create_balanced_datasets(750,test_dataset[test_dataset['Label']==0],test_dataset[test_dataset['Label']==1])
            test_dataset = clean_dataset(test_dataset)
            test_dict[dataset_name]['best_report'],test_dict[dataset_name]['best_confusion_matrix'] ,test_dict[dataset_name]['f1_1'],test_dict[dataset_name]['f1_weighted'], test_dict[dataset_name]['FP'],test_dict[dataset_name]['FN'], test_dict[dataset_name]['TN'],test_dict[dataset_name]['TP'], test_dataset['predictions'] = test_n_times_on_other_dataset(n_iter,
                                                                                                               train_dataset_name_1,
                                                                                                               train_dataset_name_2,
                                                                                                               test_dataset,
                                                                                                               #,train_rate
                                                                                                               val_rate,
                                                                                                               train_dataset_number
                                                                                                               #datasets_dict[dataset_name]['test_rate']
                                                                                                                  )

        print("test on: "+dataset_name+" Average f1 score for Label 1 is:"+str(test_dict[dataset_name]['f1_1'])+'\n')
        print("test on: " +dataset_name+" Average f1 weighted score is:"+str(test_dict[dataset_name]['f1_weighted'])+'\n')
        print("5 False Positives: {}".format(test_dict[dataset_name]['FP'][0:6]))
        print("5 False Negatives: {}".format(test_dict[dataset_name]['FN'][0:6]))
        print("5 True Positives: {}".format(test_dict[dataset_name]['TP'][0:6]))
        print("5 True Negatives: {}".format(test_dict[dataset_name]['TN'][0:6]))
        test_dataset.to_csv('test with predictions ' + dataset_name + ' trained by ' + train_dataset_name_1 + '_RM.csv', index=False)
        with open( 'dict ' + train_dataset_name_1+ '_'+train_dataset_name_2 + '_test_on_' + dataset_name + '_' + str(train_dataset_number)+'_RM.pickle', 'wb') as handle:
            pickle.dump(test_dict[dataset_name], handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    
#     with open( 'dict' + train_dataset_name+'_RM_imbalanced.pickle', 'wb') as handle:
#         pickle.dump(test_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return test_dict

In [4222]:
Offensive_Reddit_Kaggle_dict = test_for_train_dataset('Kumar','',datasets_dict,1,0.2,1,1)

train on Kumar dataset and :

test on: Razavi
train length: 2688
337
train length 0 label: 1350
train length 1 label: 1338
val length: 672
val length 0 label: 337
val length 1 label: 335
test length: 771
test length 0 label: 386
test length 1 label: 385


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'input_ids': [[101, 2023, 2003, 1037, 14324, 2944, 14924, 4818, 102, 0], [101, 2057, 2097, 2986, 1011, 8694, 1037, 14324, 2944, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


C:\Users\Dani1.OFER3090PC\.conda\envs\rm-project\lib\site-packages\transformers\tokenization_utils_base.py:2226: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


[0.99555556 1.0044843 ]

 Epoch 1 / 4
  Batch    50  of    672.
  Batch   100  of    672.
  Batch   150  of    672.
  Batch   200  of    672.
  Batch   250  of    672.
  Batch   300  of    672.
  Batch   350  of    672.
  Batch   400  of    672.
  Batch   450  of    672.
  Batch   500  of    672.
  Batch   550  of    672.
  Batch   600  of    672.
  Batch   650  of    672.

Evaluating...
  Batch    50  of    168.
  Batch   100  of    168.
  Batch   150  of    168.

Training Loss: 0.688
Validation Loss: 0.681

 Epoch 2 / 4
  Batch    50  of    672.
  Batch   100  of    672.
  Batch   150  of    672.
  Batch   200  of    672.
  Batch   250  of    672.
  Batch   300  of    672.
  Batch   350  of    672.
  Batch   400  of    672.
  Batch   450  of    672.
  Batch   500  of    672.
  Batch   550  of    672.
  Batch   600  of    672.
  Batch   650  of    672.

Evaluating...
  Batch    50  of    168.
  Batch   100  of    168.
  Batch   150  of    168.

Training Loss: 0.664
Validation Loss: 0.

In [4223]:
with open('dict Founta__test_on_Kaggle_4.pickle', 'rb') as handle:
    Founta_Kaggle_dict = pickle.load(handle)

In [4224]:
Founta_Kaggle_dict['best_confusion_matrix']

col_0,0,1
row_0,,
0,978,22
1,571,429


In [4225]:
with open('Waseem_test_on_Offensive_Reddit_new_RM_clean.pickle', 'rb') as handle:
    Waseem_OR_dict_RM = pickle.load(handle)

In [4226]:
Waseem_OR_dict_RM['best_confusion_matrix']

col_0,0,1
row_0,,
0,810,190
1,580,420


In [4227]:
def create_dataset_with_Roastme(train_dataset_name,n):
    RM_train = pd.read_csv('train_dataset_1_RoastMe_new.csv')
    RM_train = RM_train.sample(frac=0.85)
    RM_validation = pd.read_csv('validation_dataset_1_RoastMe_new.csv')
    RM_validation = RM_validation.sample(frac=0.85)
    for i in range(1,n+1):
        train_dataset = pd.read_csv('train_dataset_'+ str(i)+'_'+train_dataset_name+'.csv')
        train_dataset = pd.concat([train_dataset,RM_train])
        train_dataset = train_dataset.sample(frac=1)
        train_dataset = train_dataset.reset_index(drop=True)
        train_dataset['id'] = range(1,len(train_dataset)+1)
        train_dataset.to_csv('train_dataset_'+str(i)+'_'+train_dataset_name + '_RM.csv',index=False)
        validation_dataset = pd.read_csv('validation_dataset_'+ str(i)+'_'+train_dataset_name+'.csv')
        validation_dataset = pd.concat([validation_dataset,RM_validation])
        validation_dataset = validation_dataset.sample(frac=1)
        validation_dataset = validation_dataset.reset_index(drop=True)
        validation_dataset['id'] = range(1,len(validation_dataset)+1)
        validation_dataset.to_csv('validation_dataset_'+str(i)+'_'+train_dataset_name + '_RM.csv',index=False)

In [4228]:
def create_dataset_with_second_dataset(train_dataset_name,second_dataset_name,n, fraction):
    for i in range(1,n+1):
        Second_train = pd.read_csv('train_dataset_' + str(i)+'_'+ second_dataset_name +  '.csv')
        Second_train = Second_train.sample(frac=fraction)
        Second_validation = pd.read_csv('validation_dataset_' + str(i)+'_'+ second_dataset_name +  '.csv')
        Second_validation = Second_validation.sample(frac=fraction)
        train_dataset = pd.read_csv('train_dataset_'+ str(i)+'_'+train_dataset_name+'.csv')
        train_dataset = pd.concat([train_dataset,Second_train])
        train_dataset = train_dataset.sample(frac=0.7)
        train_dataset = train_dataset.reset_index(drop=True)
        train_dataset['id'] = range(1,len(train_dataset)+1)
        train_dataset.to_csv('train_dataset_'+str(i)+'_'+train_dataset_name +'_' +second_dataset_name + '.csv',index=False)
        validation_dataset = pd.read_csv('validation_dataset_'+ str(i)+'_'+train_dataset_name+'.csv')
        validation_dataset = pd.concat([validation_dataset,Second_validation])
        validation_dataset = validation_dataset.sample(frac=0.7)
        validation_dataset = validation_dataset.reset_index(drop=True)
        validation_dataset['id'] = range(1,len(validation_dataset)+1)
        validation_dataset.to_csv('validation_train_dataset_'+str(i)+'_'+train_dataset_name +'_' +second_dataset_name + '.csv',index=False)

In [4229]:
#create_dataset_with_second_dataset('Offensive_Reddit_new','Founta',3,0.6)

In [4230]:
def devide_sataset_to_n_train_datasets(train_dataset_full,dataset_name,n):
    #n = 5
    train_dataset_full = train_dataset_full
    train_class_len = math.floor(math.floor(len(train_dataset_full)/n)/2)
    train_current = train_dataset_full
    for i in range(n):
        train_dataset = create_balanced_datasets(train_class_len,train_current[train_current['Label']==0],train_current[train_current['Label']==1])
    #train_dataset = kaggle_dataset
        train_current = pd.merge(train_current,train_dataset, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
        train_rate = 0.8
        train_text, val_text, train_labels, val_labels = train_test_split(train_dataset['comment'], train_dataset['Label'], 
                                                                    #random_state=2018,
                                                                    test_size= 1-train_rate,
                                                                    stratify=train_dataset['Label'])
        train_labels = train_labels.reset_index(drop=True)
        train_text = train_text.reset_index(drop=True)
        val_labels = val_labels.reset_index(drop=True)
        val_text = val_text.reset_index(drop=True)
        train_dataset_current = pd.DataFrame()
        train_dataset_current['id'] = [i for i in range(1,len(train_text)+1)]
        train_dataset_current['comment'] = train_text
        train_dataset_current['Label'] = train_labels
        validataion_dataset_current = pd.DataFrame()
        validataion_dataset_current['id'] = [i for i in range(1,len(val_text)+1)]
        validataion_dataset_current['comment'] = val_text
        validataion_dataset_current['Label'] = val_labels
        train_dataset_current.to_csv('train_dataset_' + str(i+1) +'_'+ dataset_name+'.csv',index=False, encoding="utf8" )
        validataion_dataset_current.to_csv('validation_dataset_' + str(i+1)+'_' + dataset_name+'.csv',index=False, encoding="utf8" )

### 